# Proco-ALS for fast NCPD

:::{admonition} Reference
:class: tip
{cite}`Cohen2015Fast` J. E. Cohen, R. C. Farias, and P. Comon, "Fast decomposition of large nonnegative tensors," IEEE Signal Processing Letters, vol. 22, no. 7, pp. 862-866, 2015. [pdf](https://hal.archives-ouvertes.fr/hal-01069069/document)
:::

## Projected least squares for solving NNLS

When discussing nonnegative least squares algorithms, we overlooked one possible naive idea to compute an approximate solution: first compute the unconstrainted least squares solution, then project it on the nonnegative orthant. Given an NNLS problem NNLS(A,b), this means computing
```python
    x_hat = tl.solve(A,b)
    x_hat[x_hat<0] = 0
```
While this solution is not optimal (in fact there are NNLS instances for which is is arbitrarily far from the solution, see below), it can be much faster to compute in particular for large and/or structured $A$ for which highly optimized least squares solvers exist.

A naive algorithm then consists in running the solve-then-project (STP) routine as the solver for ALS to compute nonnegative CP. A pitfall to avoid is that sign ambiguity might mean columns of a factor are all negative at some iteration after the solve operation. Then the projection on the nonnegative orthant yield a zero vector which makes the next solve ill-posed. A work-around is to flip negative vectors when they occur. The obtained algorithm is coined Pro-ALS (Projected ALS).

In [ ]:
import tensorly as tl
from tensorly.tenalg import unfolding_dot_khatri_rao
from tensorly.decomposition import non_negative_parafac_hals
from tensorly_hdr.gradients import get_pseudo_inverse, err_calc_simple
from copy import deepcopy

def pro_als_basic(T, r, init, itermax = 100):
    # Input tensor T, rank of cp decomposition rank r, init is a cp_tensor
    cp_e = deepcopy(init) # estimated cp_tensor
    norm_tensor = tl.norm(T, 2) ** 2
    err = []
    for i in range(itermax): # iterative algorithm loop
        for mode in range(T.ndim): # alternating algorithm loop
            # form MTTKRP, pseudo-inverse and solve least squares
            mttkrp = unfolding_dot_khatri_rao(T, cp_e, mode)
            pseudo_inverse = get_pseudo_inverse(cp_e, r, mode, T)
            factor = tl.transpose(tl.solve(tl.transpose(pseudo_inverse), tl.transpose(mttkrp)))
            # Projection
            # if factor is full negative, flip it
            support = factor<0
            for q in range(r):
                if support[:,q].all():
                    factor[:,q] = -factor[:,q]
                else:
                    factor[:,q][support[:,q]] = 0
            cp_e[1][mode]=factor
            
        # compute error
        err.append(err_calc_simple(cp_e,mttkrp,norm_tensor))
        # BUG ?? this normalizes the tensor by default??
        #unnorml_rec_error, _, _ = error_calc(T, norm_tensor, None, cp_e[1], 0, None, mttkrp=mttkrp)
            
    return cp_e, err

We can test this algorithms for computing nonnegative CP on a simple synthetic example.

In [ ]:
# We can test it as such
rank=2
dims = [10,10,10]
cp_true = tl.random.random_cp(dims, rank) # rank 2, dim 10x10x10
cp_true[0] = None # no weights
init_cp = tl.random.random_cp(dims, rank)
init_cp[0] = None
for i in range(3):
    cp_true[1][i] = tl.maximum(cp_true[1][i],0)
    init_cp[1][i] = tl.maximum(init_cp[1][i],0)
T = cp_true.to_tensor()
cp_e, err = pro_als_basic(T, rank, init_cp)
# Comparison with tensorly nn_cp hals
err_hals = []
def err_ret(_,b):
    err_hals.append(b)
    
out2 = non_negative_parafac_hals(T, rank, init=deepcopy(init_cp), callback=err_ret)
print(err)
print(err_hals)

Use this to decompose one of the example nn tensors, see it works (ish), have to be careful.
Cite my work:{cite}`li:2019:sparse.tensor.reordering`

The cost at each Proco-ALS iteration empirically goes down although this cannot be guaranteed. The algorithm is much slower per iteration than the HALS algorithm, but the cost per iteration is significantly lower [todo show].

## Proco-ALS

### Tucker compression for faster CP-Decomposition

The true interest of Pro-ALS is when it is used in conjonction with Tucker compression. The idea of Tucker compression is to first compute an orthogonal, approximate Tucker decomposition with small inner dimensions using e.g. HOSVD, then work on the resulting smaller core tensor to compute the CP decomposition.

[insert figure]

Mathematically, this makes sense because of the associativity of the multiway tensor product:

$$ T = \left( A \otimes B \otimes C \right) I_r = \left( UA_c \otimes VB_c \otimes WC_c \right) I_r = \left(U\otimes V\otimes W \right) \left(A_c\otimes B_c\otimes C_c \right) I_r $$

where the tensor $\left(A_c\otimes B_c\otimes C_c \right) I_r \in \mathbb{R}^{r_1\times r_2\times r_3}$ can be understood as a core tensor $G$ with a CP decomposition of rank $r$ and small factors $A_c,B_c,C_c$.

In Tensorly we can perform Tucker compression (also called CANDELINC~\ref{}) easily as follows

In [ ]:
cp_true = tl.random.random_cp([100,100,100], 3)
T = cp_true.to_tensor()

# Perform Tucker on T
t_tensor = tl.decomposition.tucker(T, [3,3,3])
# Perform CP on G
G = t_tensor[0]
small_cp_est = tl.decomposition.parafac(G, 3)
# Recover larger cp by chain rule
cp_est = tl.cp_tensor.CPTensor((small_cp_est[0], [t_tensor[1][i]@small_cp_est[1][i] for i in range(T.ndim)]))

# Reconstruction error
print(tl.norm(cp_est.to_tensor() - T)/tl.norm(T))

# Comparing with direct cp algorithm
cp_est_direct = tl.decomposition.parafac(T, 3)
print(tl.norm(cp_est_direct.to_tensor() - T)/tl.norm(T))

### Tucker compression for Nonnegative CP

This procedure is standard for unconstrained CP as it offers little inconvenience when high precision is not required. It is highly recommended in the scenario where several CP decompositions of the same tensor are to be computed.

However for nonnegative CP, when working on the core tensor $G$, one has to keep in mind that nonnegativity applies on the original factors, not the small compressed ones. It means that when updating e.g. factor $A_c$ during the CP decomposition of $G$, the problem to solve is

$$ \min_{UA_c\geq 0} \|G_{[1]} - A_c\left(B_c \odot C_c \right)^T \|_F^2 $$

which is not a typical NNLS problem. It is still a quadratic program, we can solve it up to machine precision with a variety of algorithms. However these algorithms might be costly. Consider for instance a projected gradient descent algorithm

$$ A_c^{k+1} = \Pi_{U\cdot\geq 0}\left[ A_c - \eta \left( G_{[1]}\left(B_c \odot C_c\right) + \left(B_c^TB_c \ast C_c^TC_c \right)\right)\right] $$

The cost of the gradient step is low because of Tucker compression, but conversely the cost of the projection on the positive cone of $U$ can be consequent if $U$ is a large matrix (which is exactly the setup we consider for Tucker compression). This projection is in fact exactly a collection of $n_1$ NNLS problem of dimensions $r$, with $n_1$ the dimension in the first mode of the original tensor. 

In our work [ref, date], we proposed to use the Pro-ALS idea to avoid resorting to NNLS solvers entirely. This gave birth to the Proco-ALS algorithm detailed below. We first solve the least squares problem unconstrained, then project on the constraint set $UA_c\geq0$. As mentioned above, such a projection is also costly, therefore we use a heuristic approximate projection

$$ \hat{\Pi}(y) = U^T\left[Ux\right]_+ $$

It turns out that $\hat{\Pi}$ is one iteration of alternating projection on convex sets (projection on the nonnegative orthant, and on the linear span of $U$ since $U$ is a frame). In particular it is a non-expensive operator. This means that iterating $\hat{\Pi}$ several times would converge towards $\Pi$. However for performance reasons we only suggest to run it once.

In [ ]:
def proco_als(G, r, init, comp_facs, itermax=100):
    # Input tensor G, rank of cp decomposition rank r, init is a cp_tensor, comp_facs are U,V,W
    cp_e = deepcopy(init) # estimated cp_tensor
    norm_tensor = tl.norm(G, 2) ** 2
    err = []
    for i in range(itermax): # iterative algorithm loop
        for mode in range(G.ndim): # alternating algorithm loop
            # form MTTKRP, pseudo-inverse and solve least squares
            mttkrp = unfolding_dot_khatri_rao(G, cp_e, mode)
            pseudo_inverse = get_pseudo_inverse(cp_e, r, mode, G)
            factor = tl.transpose(tl.solve(tl.transpose(pseudo_inverse), tl.transpose(mttkrp)))
            
            # Projection on UA_c >= 0
            # First decompress
            factor_big = comp_facs[mode]@factor
            # if factor is full negative, flip it, otherwise project
            support = factor_big<0
            for q in range(r):
                if support[:,q].all():
                    factor_big[:,q] = -factor_big[:,q]
                else:
                    factor_big[:,q][support[:,q]] = 0
            # recompress
            factor = comp_facs[mode].T@factor_big
            cp_e[1][mode]=factor
            
        # compute error in the compressed domain
        err.append(err_calc_simple(cp_e,mttkrp,norm_tensor))
            
    return cp_e, err

Again we can test this proposed Proco-ALS algorithm on a simple synthetic dataset.

In [ ]:
cp_true = tl.random.random_cp([100,100,100], 3)
init_cp = tl.random.random_cp([100,100,100], 3)
for i in range(3):
    cp_true[1][i] = tl.maximum(cp_true[1][i],0)
    init_cp[1][i] = tl.maximum(init_cp[1][i],0)
cp_true[0] = None
init_cp[0] = None
T = cp_true.to_tensor()

# Perform Tucker on T
t_tensor = tl.decomposition.tucker(T, [3,3,3])
# Perform CP on G
G = t_tensor[0]
init_small = tl.cp_tensor.CPTensor((None,[t_tensor[1][i].T@init_cp[1][i] for i in range(T.ndim)]))
small_cp_est, err = proco_als(G, 3, init_small, t_tensor[1])
# Recover larger cp by chain rule
cp_est = tl.cp_tensor.CPTensor((small_cp_est[0], [t_tensor[1][i]@small_cp_est[1][i] for i in range(T.ndim)]))

# Reconstruction error
print(tl.norm(cp_est.to_tensor() - T)/tl.norm(T))

# Comparing with direct cp algorithm
cp_est_direct = tl.decomposition.non_negative_parafac_hals(T, 3)
print(tl.norm(cp_est_direct.to_tensor() - T)/tl.norm(T))

```{admonition} Note on the Projection $\Pi$

We use KKT to show that solving

$$ \min_{Ux\geq 0} \frac{1}{2} \|x - \hat{x}\|_2^2 $$

is equivalent to an NNLS problem

$$ \min_{z\geq 0} \frac{1}{2} \|U^Tz + \hat{x}\|_2^2 $$

This equivalence also has a geometric interpretation: we either project $-\hat{x}$ on the dual cone of $U$ (second problem) or find the closest element to $\hat{x}$ in the intersection of half planes (first problem). 

[insert figure]

The Lagrangian for the projection problem writes $L(x,\mu) = \frac{1}{2}\|x - \hat{x} \|_2^2  - \mu^TUx$
which when minimized w.r.t. $x$ yields $x^\ast = \hat{x} + U^T\mu^\ast$.

The dual problem is therefore formalized as 
$ \min_{\mu\geq 0} -\frac{1}{2}\|U^T\mu\|_2^2 + \|U^T\mu\|_2^2 + \mu^TU\hat{x}$ which has the same minimizer than $\min_{z\geq 0} \frac{1}{2} \|U^Tz + \hat{x}\|_2^2$

To summarize, we may compute the projection $\Pi_{U\cdot}(y)$ by first solving several small NNLS problems with mixing matrix $U^T$. This yields $z$. Then we compute the projection by using the primal-dual relationship $\Pi_{U\cdot}(y) = y + U^Tz$.
```

```{admonition} Note on the suboptimality of projected least squares estimates
Using $\left[A^\dagger b\right]_+$ as the solution to an NNLS problem $\min_{x\geq 0} \|Ax - b\|_2^2$ can be a terrible idea for some problem instances. We can use numpy to generate examples where projected least squares solutions are arbitrarily far from the true NNLS solutions, even in two dimensions. This may happen in particular when the linear system is poorly conditionned. In the plot below, the projected least squares solution is always zero, but the NNLS solution can be made arbitrarily large by stretching and rotating the mixing matrix $A$ as desired.
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Generate data
grid_x = np.meshgrid(np.linspace(-1,1,50),np.linspace(-2,2,50))
grid_z = np.copy(grid_x[0])
theta = 3.14/12
A = np.array([[1,0],[0,0.01]])@np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]]) #scale and rotate
x_LS = np.array([-0.5,-0.2])
b = A@x_LS

# LS, Pro-LS, NNLS sols
x_pro_LS = np.maximum(x_LS,0)
x_NNLS = tl.solvers.nnls.active_set_nnls(A.T@b,A.T@A)

# Plot 
for i in range(grid_z.shape[0]):
    for j in range(grid_z.shape[1]):
        grid_z[j,i] = np.linalg.norm(b - A@np.array([grid_x[0][0][i],grid_x[1][j][0]]))
plt.contourf(grid_x[0], grid_x[1], grid_z)
plt.plot([0,1],[0,0],'r')
plt.plot([0,0],[0,2],'r')
plt.scatter(x_LS[0],x_LS[1]) # Least squares solution is blue dot
plt.scatter(x_pro_LS[0],x_pro_LS[1]) # Pro-LS is orange dot (zero)
plt.scatter(x_NNLS[0], x_NNLS[1]) # NNLS solution is green dot
plt.colorbar()
plt.show()

Color stand for the cost $\|Ax-b\|_2$. Red lines mark the nonnegative orthant. Solution to the LS problem is at the center of the darkest ellipse.